In [ ]:
import pandas as pd 
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
df=pd.read_csv('rating.csv')

In [ ]:
df.head()

In [ ]:
rating_pt=df.pivot(index='userId',columns='movieId',values='rating').fillna(0)

In [ ]:
rating_pt

## Breaking data set into trainig and testing data set.
* 80% for training data.
* 20% for testing data.

In [ ]:
train_rating_data=df[0:80000]

In [ ]:
test_rating_data=df[80000:100000]

In [ ]:
R_known_train=train_rating_data.rating.values


In [ ]:
R_known_train

In [ ]:
R_known_test=test_rating_data.rating.values

In [ ]:
R_known_test

In [ ]:
user_indices_train=[i-1 for i in train_rating_data.userId.values]
movie_indices_train=[i-1 for i in train_rating_data.movieId.values]

In [ ]:
user_indices_test=[x-1 for x in test_rating_data.userId.values]
movie_indices_test=[x-1 for x in test_rating_data.movieId.values]


## Preparing data

In [ ]:
R=rating_pt
N=610 #no.of users
M=9724 #no of items
K=10 #no of hidden features
p=np.random.rand(N,K)
q=np.random.rand(K,M)

## In Matrix factorization prediction is eqaul to-
           prediction: r^=pq

In [ ]:

#Prdection matrix
P=tf.Variable(p, dtype=tf.float32,name='P')
Q=tf.Variable(q,dtype=tf.float32,name='Q')
R_hat=tf.matmul(P,Q)

In [ ]:
R_hat

In [ ]:
#for cost
R_hat_flatten=tf.reshape(R_hat,[-1])
R_hat_train_known=tf.gather(R_hat_flatten,user_indices_train*tf.shape(R_hat)[1]+
                            movie_indices_train,name='extracting_user_rate')

## Cost: ||r^-r||

In [ ]:
dif_rating=tf.subtract(R_hat_train_known,R_known_train,name='diff_rating')
dif_rating_abs=tf.abs(dif_rating,name='dif_rating_abs')
base_cost=tf.reduce_sum(dif_rating_abs,name='sum_abs_error')

In [ ]:
base_cost

# Regularization:
## Σλ(||p||+||q||)

In [ ]:
lda=tf.constant(.001,name='lambda')
norm_sums=tf.add(tf.reduce_sum(tf.abs(P,name='user_abs'),name='user_norm'),
                tf.reduce_sum(tf.abs(Q,name='movies_abs'),name='movies_norm'))
regularization=tf.multiply(norm_sums,lda,'regularization')

In [ ]:
regularization

# Total Cost:  Σ||r^-r||+λ(||p||+||q||)

In [ ]:
#total cost
cost=tf.add(base_cost,regularization)

In [ ]:
#creating a optimizer to minimize the loss
optimizer=tf.train.GradientDescentOptimizer(0.001)
train=optimizer.minimize(cost)

In [ ]:
sess=tf.Session();
init=tf.global_variables_initializer()
sess.run(init)
for i in range(500):
    sess.run(train)

In [ ]:
R_hat_test_known=tf.gather(R_hat_flatten,user_indices_test*tf.shape(R_hat)[1]+
                           movie_indices_test,name='extracting_user_rate')

In [ ]:
R_hat

In [ ]:
dif_rating_test=tf.subtract(R_hat_test_known,R_known_test,name='dif_rating_test')
dif_rating_abs_test=tf.abs(dif_rating_test)
sum_dif_rating_abs_test=tf.reduce_sum(dif_rating_abs_test)
accuarcy_test=tf.div(sum_dif_rating_abs_test,R_known_test.size)

In [ ]:
accuarcy_test

In [ ]:
R.head()

## VIsualize R_hat rating prediction ,to compare with known R

In [ ]:
pred=sess.run(R_hat)
pred_df=pd.DataFrame(pred)
pred_df.head()